# Lab04 Data Analysis: Introduction to 'pandas'
**Objective:**

Through this assignment, students will become familiar with basic operations using pandas.

## About Dataset

**Context**

**note**

You can use pipenv to install all modules you need

In [1]:
# import lybrary modules
import pandas as pd
import math
from pandas.api.types import is_string_dtype
import numpy as np
import matplotlib.pyplot as plt

# Problem B pokemon (30 points)

In [2]:
# read 'Pokemon.csv' file
df = pd.read_csv('Pokemon.csv', delimiter=',', encoding='utf-8')
display(df.head())

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## Question 1 (10 pts)

a. How many percentage of legendary Pokemons with an `Attack` value greater than 150? (3 pts) 

Also, we want to know how many percentage of non-legendary Pokemons with an `Attack` value greater than 150? (3 pts)

Describe what you found. (1 pts)

b. With the scatter plot below, find which pokemon is the outlier at the lower right corner. (3 pts)


In [4]:
def legendary_attack(df, is_legendary=True):
    """

    Args:
        df (pd.DataFrame): pokemon dataframe

    Returns:
        pd.DataFrame: the whole state of the legendary pokemon with the highest attack
    """
    
    if is_legendary:
        legendary_pokemon = df.loc[df['Legendary']]
        number_all = legendary_pokemon['Name'].count()
        number_over150 = legendary_pokemon.loc[legendary_pokemon['Attack'] > 150]['Name'].count()
        ans = 'The percentage of legendary pokemon with attack over 150 is {:.1f}%. '.format(number_over150 / number_all*100)
        return ans
    else:
        non_legendary_pokemon = df.loc[~df['Legendary']]
        number_all = non_legendary_pokemon['Name'].count()
        number_over150 = non_legendary_pokemon.loc[non_legendary_pokemon['Attack'] > 150]['Name'].count()
        ans = 'The percentage of non-legendary pokemon with attack over 150 is {:.1f}%. '.format(number_over150 / number_all*100)
        return ans


print(legendary_attack(df, True))
print(legendary_attack(df, False))

The percentage of legendary pokemon with attack over 150 is 12.3%. 
The percentage of non-legendary pokemon with attack over 150 is 1.4%. 


In [5]:
df_defense = df.loc[df['Defense'] >= 200]
defense_attack = df_defense.sort_values(by=['Attack'], ascending=True)
display(defense_attack)

# Answer: Shuckle

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,2,False
223,208,Steelix,Steel,Ground,510,75,85,200,55,65,30,2,False
414,377,Regirock,Rock,NaN,580,80,100,200,50,100,50,3,True
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,2,False
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,3,False


## Question 2 (10 pts)
We have a DataFrame about pokemons now. We want to know the distribution of pokemon for each type.

Complete the function `pokemon_type_count()`


**note**

We only consider type1.

In [7]:
def pokemon_type1_count(pokemon):
    """
    compute the number of pokemons for each type1
    

    Args:
        pokemon (pd.DataFrame) : pokemon dataframe
        
    Returns:
        Dict[str, int] : dictionary of pokemon types and their counts
    """
    
    ans = {}
    types = pokemon['Type 1'].unique()
    for type in types:
        ans[type] = pokemon[pokemon['Type 1'] == type].shape[0]
        
    return ans

In [8]:
def test_pokemon_type(pokemon_dict):
    """
    test the pokemon type dictionary
    
    Args:
        pokemon_dict (Dict[str, int]) : pokemon type dictionary
        
    Returns:
        None
    """
    
    assert isinstance(pokemon_dict, dict), 'pokemon_dict should be a dictionary'
    assert all(isinstance(key, str) for key in pokemon_dict.keys()), 'pokemon_dict keys should be strings'
    assert all(isinstance(value, int) for value in pokemon_dict.values()), 'pokemon_dict values should be integers'
    assert all(value > 0 for value in pokemon_dict.values()), 'pokemon_dict values should be positive integers'
    assert pokemon_dict['Water'] == 112, 'pokemon_dict should have 112 water pokemon'
    assert pokemon_dict['Normal'] == 98, 'pokemon_dict should have 98 normal pokemon'
    assert sum(pokemon_dict.values()) == 800, 'pokemon_dict values should sum to 800'
    
    print('All tests passed!')


test_pokemon_type(pokemon_type1_count(df))


All tests passed!


## Question 3 Average attack value in different type (10 pts)

We want to compare the attack value in each generation and visualize it as a horizontal bar chart.

(a). Complete the function `average_attack_type()` (5 pts)

(b). Visualize the data with horizontal bar chart. (5 pts)


**note**
* If the pokemon has multiple types, you should consider it in both types.

In [9]:
def average_attack_type(pokemon):
    """
    compute the average attack for each type1
    

    Args:
        pokemon (pd.DataFrame) : pokemon dataframe
        
    Returns:
        Dict[str, int] : dictionary of pokemon types and their average attack
    """
    
    ans = {}
    types = pokemon['Type 1'].append(pokemon['Type 2']).dropna().unique()
    
    for type in types:
        ans[type] = pokemon[(pokemon['Type 1'] == type) | (pokemon['Type 2'] == type)]['Attack'].mean()
        
    return ans

In [11]:
def test_average_attack_type(average_attack_type):
    """
    test the average attack type dictionary
    
    Args:
        average_attack_type (Dict[str, int]) : average attack type dictionary
        
    Returns:
        None
    """
    assert isinstance(average_attack_type, dict), 'average_attack_type should be a dictionary'
    assert all(isinstance(key, str) for key in average_attack_type.keys()), 'average_attack_type keys should be strings'
    assert all(isinstance(value, float) for value in average_attack_type.values()), 'average_attack_type values should be floats'
    assert all(value > 0 for value in average_attack_type.values()), 'average_attack_type values should be positive floats'
    assert np.allclose(average_attack_type['Water'], 73.7063492063492), 'average_attack_type should have 73.7063492063492 water pokemon'
    assert np.allclose(average_attack_type['Grass'], 73.46315789473684), 'average_attack_type should have 73.46315789473684 grass pokemon'
    
    print('All tests passed!')
    
    
    

test_average_attack_type(average_attack_type(df))

All tests passed!


/var/folders/6j/s6_hc9510lzglvgg014gz3wh0000gn/T/ipykernel_18716/348577800.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  types = pokemon['Type 1'].append(pokemon['Type 2']).dropna().unique()
